In [5]:
import os
import re
import pandas as pd

In [3]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')

train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

In [4]:
train

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [10]:
# 이상한 문장인지 판단
def is_abnormal(text):
    try:
        if not isinstance(text, str) or len(text.strip()) == 0:
            return True
            
        # 공백 제거 (길이 대비 비율 도출 위함)
        text_no_space = text.replace(" ", "")
        total_len = len(text_no_space)
        
        # 문자 유형별 개수 및 비율
        hangul_ratio = len(re.findall(r'[가-힣]', text_no_space)) / total_len
        special_chars = re.findall(r'[^가-힣a-zA-Z0-9\s,.…"\'%↑↓→←]', text_no_space)
        special_ratio = len(special_chars) / total_len
        
        # 비정상 패턴
        abnormal_patterns = re.findall(r'[A-Za-z][0-9]|[0-9][A-Za-z]|[A-Za-z][^A-Za-z0-9\s]|[^A-Za-z0-9\s][A-Za-z]', text)
        pattern_ratio = len(abnormal_patterns) / total_len
        
        # Threshold
        conditions = [
            special_ratio > 0.15,                    # 특수문자 비율
            hangul_ratio < 0.4,                      # 한글 비율
            pattern_ratio > 0.1,                     # 비정상 패턴 비율
            bool(re.search(r'[A-Za-z]\d[A-Za-z]', text)),  # 알파벳-숫자-알파벳 패턴
            bool(re.search(r'[^가-힣a-zA-Z0-9\s]{2,}', text)),  # 연속된 특수문자
            len(set(special_chars)) > 2              # 다양한 종류의 특수문자
        ]
        
        return any(conditions)
        
    except Exception as e:
        print("Error:", e)
        return True

In [11]:
train['abnormal'] = train['text'].apply(is_abnormal)

print(f"이상으로 판단된 train의 수 : {len(train[train.abnormal])}\n")
train

이상으로 판단된 train의 수 : 1616



,ID,text,target,abnormal
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,True
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,True
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,True
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,False
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,True
...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,False
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,False
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4,True
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,False


In [13]:
train[~train.abnormal]

,ID,text,target,abnormal
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,False
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,False
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6,False
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,False
11,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1,False
...,...,...,...,...
2794,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2,False
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,False
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,False
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,False
